In [7]:
from app import *
import pyperclip as clp

window_width = 2400
top_banner_height = 62
bottom_window_height = 1491
hao_height = 144
hao_width = 416
hao_sub_width = 704 # 号的文章栏的宽度 貌似没啥用
article_2rd_height = 196 # 除头条外文章的高度
v_first_center = top_banner_height + hao_height / 2
h_first_center = 66

def find_my1():
    my1v = img_loc("images_mac/my1_mac.png")
    return my1v

def find_my2():
    my2v=img_loc("images2/my2_2.png")
    return my2v

def img_loc(img):
    my=CardLocation(config=CardConfig(template_file=img, bias_type=BiasType.NoBias))
    myv=my.get_location
    return myv

def moveby(mv, x, y):
    time.sleep(0.8)
    pag.moveTo(x=mv.x+x,y=mv.y+y)
    time.sleep(0.8)

def close_dingyuehao():
    try:
        my1v = find_my1()
        moveby(my1v, 2, 2)
        pag.click()
        pag.hotkey('alt', 'f4')
    except:
        pass

def re_open_dingyuehao():
    pag.hotkey('ctrl','alt', 'w')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('esc')
    time.sleep(0.2)
    pag.hotkey('ctrl','alt', 'w')
    pag.hotkey('win', 'up')
    pag.moveTo(x = 190, y = 66)
    pag.click()
    clp.copy("订阅号")
    time.sleep(0.2)
    pag.hotkey('ctrl', 'a')
    time.sleep(0.2)
    pag.hotkey('ctrl', 'v')
    pag.moveTo(x = 166, y = 232)
    pag.click()
    pag.moveTo(x = 176, y = 176)
    pag.click(clicks=2, interval=0.1, button='right')
#     moveby(mpls, 0, 0)
#     pag.click(clicks=2, interval=0.1, button='right')#最大化
    time.sleep(0.3)
    pag.hotkey('win', 'up')
# close_dingyuehao()
# re_open_dingyuehao()


In [ ]:
# mpls = img_loc("images_mac/main_plus.png")

In [12]:
from openai_tools import *
from pachong2 import *

from db import SqliteDb

def locate_hao(i=0):
    pag.moveTo(x = 66, y = 58+66+ i*144)
    if i == 0:
        pag.scroll(10000000)
    time.sleep(0.3)
    pag.click()
    
def open_curr_article_to_browser():
    my1v=find_my1()
    pag.moveTo(x = 2370, y = 58+47)
    pag.click()
    pag.moveTo(x = 2360, y = 58+47+200)
    pag.click()

def copy_browser_url():
    time.sleep(1.2)
    pag.hotkey('ctrl', 'l') 
    pag.hotkey('ctrl', 'c') 
    url = clp.paste()
    time.sleep(0.3)
    pag.hotkey('ctrl', 'w') 
    return url

def fetch_url_ctt(url):
    print(url)
    if db.check_url(url):
        hao, title, desc, abst, ctt = db.read('article', columns=' hao, title, desc, abst, ctt ', where=f"url='{url}'")[0]
    else:
        hao, title, desc, ctt = get_wechat_artile_content(url)
        if hao is None:
            return None, None, None, None, None
        abst = ''
        if ctt:
            for i in range(4):
                try:
                    _, abst = call_with_sys(sys_prompt, ctt[:6500], 512)
                    if abst:
                        time.sleep(20)
                        break
                except openai.error.APIConnectionError:
                    print('except error.APIConnectionError, sleep...')
                    time.sleep(3)
        db.create('article', {
            'title': title,
            'hao': hao,
            'url': url,
            'desc': desc,
            'abst': abst,
            'ctt': ctt
        }) 
    print("%s : %s" % (hao, title))
    return hao, title, desc, abst, ctt

def write2file(i, hao, title, url, desc, abst, ctt):
    if i == 0:
        f_meta.write("## %s \n\n" % hao)
        f_meta.flush()
    f_meta.write("### %s \n\n %s\n\n %s\n\n" % (title, abst, url))
    f_meta.flush()

    f_ctt.write("%s : %s \n %s\n%s\n\n" % (hao, title, desc, url))
    f_ctt.write("%s\n\n" % abst)
    f_ctt.write("%s\n\n\n\n\n\n" % ctt)
    f_ctt.flush()

def click_curr_hao_all_article(num):
    my1v=find_my1()
    i = -1
    while True:
        i += 1
        if i < num:
            moveby(my1v, hao_width + 30, top_banner_height + 400 + i * article_2rd_height)
            pag.click()
            time.sleep(1.3)
            open_curr_article_to_browser()
            url = copy_browser_url()
            if url not in all_ctt:
                hao, title, desc, abst, ctt = fetch_url_ctt(url)
                all_ctt[url] = title
                if hao is not None:
                    write2file(i, hao, title, url, desc, abst, ctt)
        else:
            break
    f_meta.write("<hr/>\n\n\n\n\n\n")
    f_meta.flush()

# open_curr_article_to_browser()

In [14]:
def wechat_article_list(hao_num, art_num):
    for i in range(hao_num + 1):
#         my1v=find_my1()
        if i == 0:
            locate_hao()
        elif 1 <= i <= 9:
#             moveby(my1v, h_first_center, v_first_center + hao_height * i)
            locate_hao(i)
            pag.click()
        else:
#             moveby(my1v, h_first_center, v_first_center + hao_height * 9)
            locate_first(9)
            pag.click()
            pag.hotkey('down')
        time.sleep(0.3)
        click_curr_hao_all_article(art_num)

if __name__ == "__main__":
    sys_prompt = "跟据用户给出的文章提取80-120字的摘要，具体时间、人物等信息不可以省略。"
    while True:
        db = SqliteDb('test.sqlite')
        db.create_table('article', 'hao TEXT, title TEXT, url TEXT, desc TEXT, abst TEXT, ctt TEXT')

        now = datetime.now().strftime('%Y%m%d_%H%M%S')
        f_meta = open("_wechat_url" + "_" + now + ".md", "w", encoding="utf-8")
        f_ctt = open("_wechat_content" + "_" + now + ".txt", "w", encoding="utf-8")

        all_ctt = {}
        wechat_article_list(12, 1)
        f_meta.close()
        f_ctt.close()
        db.close()
        
        time.sleep(10 * 60)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'